In [3]:
from glob import glob
import json
import math
import os

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

## Data

In [6]:
# Pathing
# =======
BDD_DIR = os.environ['HOME'] + '/.Data/BDD/bdd100k'
DET_LABELS_DIR = BDD_DIR + '/labels/detection20'


# Files
# =====
DET_LABELS_TRAIN = DET_LABELS_DIR + '/det_v2_train_release.json'
DET_LABELS_VAL   = DET_LABELS_DIR + '/det_v2_val_release.json'

In [7]:
def read_json(path):
    with open(path) as jfile:
        return json.load(jfile)

labels_train = read_json(DET_LABELS_TRAIN)
labels_val   = read_json(DET_LABELS_VAL)

In [10]:
print(f"{len(labels_train)} {len(labels_val)}")

69863 10000


In [8]:
labels_train[0]

{'videoName': '0000f77c-6257be58',
 'name': '0000f77c-6257be58.jpg',
 'index': 0,
 'timestamp': 10000,
 'labels': [{'category': 'traffic light',
   'poly2d': None,
   'box3d': None,
   'box2d': {'x2': 1156.978645,
    'y1': 133.184488,
    'x1': 1125.902264,
    'y2': 210.875445},
   'manualShape': True,
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': [1, 'G']},
   'id': 0},
  {'category': 'traffic light',
   'poly2d': None,
   'box3d': None,
   'box2d': {'x2': 1191.50796,
    'y1': 136.637417,
    'x1': 1156.978645,
    'y2': 210.875443},
   'manualShape': True,
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': [1, 'G']},
   'id': 1},
  {'category': 'traffic sign',
   'poly2d': None,
   'box3d': None,
   'box2d': {'x2': 1170.79037,
    'y1': 211.122087,
    'x1': 1105.66915985699,
    'y2': 233.566141},
   'manualShape': True,
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': 